# Example ingest and data cleanse

In this example we ingest publically available Amazon rating data for purposes of SVD analisys.  We also do some additional filtering to reduce the dataset to a manageable size.

### Steps

1. Download compressed file.
1. Read file to DataFrame.
1. Create a new dataframe with the columns we need. 
1. Do some data cleansing.
1. Save to `amazon_reviews_in` table.
1. Create `amazon_customers` table.
1. Create `amazon_products` table.
1. Create `amazon_matrix` table.

### See Also

* [Amazon Customer Reviews Dataset](https://s3.amazonaws.com/amazon-reviews-pds/readme.html)
* [Dataset Index](https://s3.amazonaws.com/amazon-reviews-pds/tsv/index.txt)

In [1]:
%%html
<style>
  table {margin-left: 0 !important;}
</style>

### Generated Tabeles

We do some filtering on the `amazon_reviews_in` to reduce the size of data for SVD.

| Table Name | Description |
| ---: | ---: |
| amazon_reviews_in | Raw ingested data. |
| amazon_customers | Customers who have rated more than 3 products. |
| amazon_products | Top 500 rated products. |
| amazon_matrix | Filtered rating data that will be used for SVD. |

In [2]:
# Local libraries should automatically reload
%reload_ext autoreload
%autoreload 1

import sys 
sys.path.append('../KJIO')
import pandas as pd
import numpy as np
%aimport kapi_io
import gpudb

# create tables in this schema
SCHEMA = 'SVD'

_tsv_file = 'amazon_reviews_us_Tools_v1_00.tsv.gz'

### Download file

This data contains 1,741,100 ratings for 168,648 unique products for the tools category. Other product categories are available.

File Source: [amazon_reviews_us_Tools_v1_00.tsv.gz](https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Tools_v1_00.tsv.gz) (333.8 MB)

Columns:

```
marketplace       - 2 letter country code of the marketplace where the review was written.
customer_id       - Random identifier that can be used to aggregate reviews written by a single author.
review_id         - The unique ID of the review.
product_id        - The unique Product ID the review pertains to. In the multilingual dataset the reviews
                    for the same product in different countries can be grouped by the same product_id.
product_parent    - Random identifier that can be used to aggregate reviews for the same product.
product_title     - Title of the product.
product_category  - Broad product category that can be used to group reviews 
                    (also used to group the dataset into coherent parts).
star_rating       - The 1-5 star rating of the review.
helpful_votes     - Number of helpful votes.
total_votes       - Number of total votes the review received.
vine              - Review was written as part of the Vine program.
verified_purchase - The review is on a verified purchase.
review_headline   - The title of the review.
review_body       - The review text.
review_date       - The date the review was written.
```

In [3]:
import urllib

_site = 'https://s3.amazonaws.com/amazon-reviews-pds/tsv'
_url = '{}/{}'.format(_site, _tsv_file)

print('Fetching data from {}. Please be patient...'.format(_url))
urllib.request.urlretrieve(_url, _tsv_file)
print('All done!')

Fetching data from https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Tools_v1_00.tsv.gz
All done!


### Read TSV file to DataFrame

In [5]:
import csv

print('Generating dataframe from {}. Please be patient...'.format(_tsv_file))
_tools_df = pd.read_csv(_tsv_file,
                        delimiter='\t',
                        encoding='utf-8',
                        compression='gzip',
                        error_bad_lines=True,
                        warn_bad_lines=True,
                        nrows=None,
                        low_memory=True,
                        quoting=csv.QUOTE_NONE,
                        header=0)

display('Completed with shape: {}'.format(_tools_df.shape))
display(_tools_df.dtypes)

Generating dataframe from amazon_reviews_us_Tools_v1_00.tsv.gz. Please be patient.


'Completed with shape: (1741100, 15)'

marketplace          object
customer_id           int64
review_id            object
product_id           object
product_parent        int64
product_title        object
product_category     object
star_rating           int64
helpful_votes         int64
total_votes           int64
vine                 object
verified_purchase    object
review_headline      object
review_body          object
review_date          object
dtype: object

### Create a new dataframe with less columms

In [5]:
_tools_new = pd.concat([
    _tools_df['customer_id'], 
    _tools_df['product_id'],
    _tools_df['product_parent'],
    _tools_df['product_title'],
    _tools_df['star_rating']
    ], 
    axis=1).set_index('customer_id')

# Display contents
_tools_new.head()

,product_id,product_parent,product_title,star_rating
customer_id,,,,
15785389,B00H5U9ZD6,115362950,WallPeg 12 sq ft Black Workbench Pegboard Orga...,5
47910848,B001TJGCS0,570955425,Nite Ize Nite Dawg Light Up Dog Collar,4
36328996,B000NIK8JW,128843593,Stanley 84-058 4 Piece Bi-Metal Pliers SetQty ...,1
51785809,B008ZYRGUA,407828107,Powerextra 14.4V Replacement Battery For Ryobi...,4
40757491,B00K5CA0GC,490746675,Waterproof Invisible Fence® Repair Splice Kit,5


### Do some misc data cleansing

In [6]:
# find out max size of product_title
_max_product = _tools_new['product_title'].str.len().max()
display('product_title max: {}'.format(_max_product))

# find max size of product_id
product_id = _tools_new['product_id'].str.len().max()
display('product_id max: {}'.format(product_id))

# trim to 100 chars
_tools_new['product_title'] = _tools_new['product_title'].map(lambda x: x[:100])

# remove unicode
_tools_new['product_title'] = _tools_new['product_title'].map(lambda x: x.encode('ascii', 'ignore').decode('ascii'))

# convert long to int
_tools_new['star_rating'] = _tools_new['star_rating'].astype(np.int32)

display('Done!')

'product_title max: 400'

'product_id max: 10'

'Done!'

### Save dataframe to amazon_reviews_in table.

In [7]:
## Override default string size of 16
_col_props = { 'product_id' : [gpudb.GPUdbColumnProperty.CHAR16],
               'product_title' : [gpudb.GPUdbColumnProperty.CHAR128] }

kapi_io.save_df(_df=_tools_new, _table_name='amazon_reviews_in', _schema=SCHEMA, _col_props=_col_props)

Dropping table: <amazon_reviews_in>
Creating  table: <amazon_reviews_in>
Column 0: <customer_id> (long) ['shard_key']
Column 1: <product_id> (string) ['char16']
Column 2: <product_parent> (long) []
Column 3: <product_title> (string) ['char128']
Column 4: <star_rating> (int) []
Inserted rows into <SVD.amazon_reviews_in>: 1741100


### Create amazon_customers table

Customers who have rated at least 3 products.

In [9]:
%aimport kodbc_io
kodbc_io.execute('''
create or replace table {}.amazon_customers as (
    select 
        customer_id,
        count(1) rating_count
    from amazon_reviews_in
    group by customer_id
    having count(1) >= 3
)
'''.format(SCHEMA))

Connected to GPUdb ODBC Server (6.2.0.17.20180825221415)


0

### Create amazon_products table

Top 500 rated products.

In [10]:
%aimport kodbc_io
kodbc_io.execute('''
create or replace replicated table {}.amazon_products as (
    select top 500
        product_parent,
        product_title,
        count(1) rating_count
    from amazon_reviews_in ar
    group by product_parent, product_title
    order by rating_count desc
)
'''.format(SCHEMA))

Connected to GPUdb ODBC Server (6.2.0.17.20180825221415)


0

### Crate amazon_matrix table

Create a table of ratings where the customers and products meet the above criteria.

In [11]:
kodbc_io.execute('''
create or replace table {}.amazon_matrix as (
    select
        ar.customer_id,
        ar.product_parent,
        ar.star_rating
    from amazon_reviews_in ar
    join amazon_customers ac
        on ar.customer_id = ac.customer_id
    join amazon_products ap
        on ar.product_parent = ap.product_parent
    order by ar.customer_id
)
'''.format(SCHEMA))

Connected to GPUdb ODBC Server (6.2.0.17.20180825221415)


0